In [1]:
import numpy as np
import pandas as pd
import requests

In [91]:
column_renames = {
  "notering": "positie",
  "nr.": "positie",
  "nr": "positie",
  "jaar": "jaartal",
  "year": "jaartal",
  "title": "titel",
  "artist": "artiest"
}

In [92]:
folder = "data"
full = pd.DataFrame()
start_year = 1999
end_year = 2024

for year in range(start_year, end_year+1):
    df = pd.read_excel(f'{folder}/TOP-2000-{year}.xlsx')
    if len(df) > 2000:
        df = df.iloc[1:]
    df.columns = [str.lower(col) for col in df.columns]
    df.rename(columns=column_renames, inplace=True)
    print(df.columns.values)
    df['lijst_jaar'] = year
    df['positie'] = df['positie'].astype(int)
    df['jaartal'] = df['jaartal'].astype(int)
    df['lijst_jaar'] = df['lijst_jaar'].astype(int)
    full = pd.concat([full, df], ignore_index=True)
full

['positie' 'titel' 'artiest' 'jaartal']
['positie' 'titel' 'artiest' 'jaartal']
['positie' 'titel' 'artiest' 'jaartal']
['positie' 'titel' 'artiest' 'jaartal']
['positie' 'titel' 'artiest' 'jaartal']
['positie' 'titel' 'artiest' 'jaartal']
['positie' 'titel' 'artiest' 'jaartal']
['positie' 'titel' 'artiest' 'jaartal']
['positie' 'titel' 'artiest' 'jaartal']
['positie' 'titel' 'artiest' 'jaartal']
['positie' 'titel' 'artiest' 'jaartal']
['positie' 'titel' 'artiest' 'jaartal']
['positie' 'titel' 'artiest' 'jaartal']
['positie' 'titel' 'artiest' 'jaartal']
['positie' 'titel' 'artiest' 'jaartal']
['positie' 'titel' 'artiest' 'jaartal']
['positie' 'titel' 'artiest' 'jaartal']
['positie' 'titel' 'artiest' 'jaartal']
['positie' 'titel' 'artiest' 'jaartal']
['positie' 'titel' 'artiest' 'jaartal']
['positie' 'titel' 'artiest' 'jaartal']
['positie' 'titel' 'artiest' 'jaartal']
['positie' 'titel' 'artiest' 'jaartal']
['positie' 'titel' 'artiest' 'jaartal']
['positie' 'titel' 'artiest' 'jaartal']


,positie,titel,artiest,jaartal,lijst_jaar
0,1,Bohemian Rhapsody,Queen,1975,1999
1,2,Hotel California,Eagles,1977,1999
2,3,Child In Time,Deep Purple,1972,1999
3,4,Stairway To Heaven,Led Zeppelin,1971,1999
4,5,Paradise By The Dashboard Light,Meat Loaf,1978,1999
...,...,...,...,...,...
51995,1996,Once Upon A Time In The West,Dire Straits,1979,2024
51996,1997,Bobby Jean,Bruce Springsteen,1984,2024
51997,1998,De Langste Nacht,Goldband,2021,2024
51998,1999,It's The End Of The World As We Know It (Album...,R.E.M.,1987,2024


In [93]:
song_example, artist_example = full['titel'][0], full['artiest'][0]
song_example, artist_example

('Bohemian Rhapsody', 'Queen')

In [94]:
no_duplicates = full[["titel", "artiest"]].drop_duplicates()
len(no_duplicates), len(full)

(5581, 52000)

In [10]:
from dotenv import load_dotenv
import os

load_dotenv()
API_KEY = os.getenv("API_KEY")

In [16]:
base_url = "http://ws.audioscrobbler.com/2.0/"

def get_metadata(song, artist):
    try:
      # Make the request
      params = {
        "method": "track.getInfo",
        "api_key": API_KEY,
        "artist": artist,
        "track": song,
        "format": "json"
      }

      response = requests.get(
          base_url,
          params=params,
      )

      # Raise an error for bad status codes
      response.raise_for_status()
      data = response.json()
      return data

    except requests.exceptions.RequestException as e:
      print(f"Request failed: {str(e)}")

In [34]:
def get_relevant_info(i: dict) -> dict:
    if i is None:
        return None
    t = i.get("track")
    if t is None:
        return None
    
    max_tags = 3
    
    return {
        "duration": t.get("duration"),
        "album": t.get("album", {}).get("title"),
        "tags": [tag["name"] for tag in t.get("toptags", {}).get("tag", [])[:max_tags]],
    }

In [35]:
first = no_duplicates.iloc[1]
data = get_metadata(first['titel'], first['artiest'])
rel_data = get_relevant_info(data)
rel_data

In [72]:
from tqdm import tqdm
full_df_list = []

In [75]:
missing_indexes = []

In [85]:
start_pos = 4000
end_pos = len(no_duplicates)

for i in tqdm(range(start_pos, end_pos)):
    track = no_duplicates.iloc[i]
    track_name = track['titel']
    artist_name = track['artiest']
    data = get_metadata(track_name, artist_name)
    rel_data = get_relevant_info(data)
    if rel_data is not None:
        rel_data["artist"] = artist_name
        rel_data["song"] = track_name
        full_df_list.append(rel_data)
    else:
        print(f"Could not find data for {track_name} by {artist_name}")
        missing_indexes.append(i)

len(full_df_list)

  3%|▎         | 48/1581 [00:10<05:17,  4.83it/s]

Could not find data for Let It Happen  by Tame Impala


  3%|▎         | 51/1581 [00:11<06:07,  4.16it/s]

Could not find data for Buddy Holly   by Weezer 


  4%|▍         | 62/1581 [00:13<05:15,  4.81it/s]

...

100%|█████████▉| 1579/1581 [05:38<00:00,  5.34it/s]

Could not find data for Fame (Albumversie) by David Bowie


100%|██████████| 1581/1581 [05:39<00:00,  4.66it/s]

Could not find data for It's The End Of The World As We Know It (Albumversie) by R.E.M.


6948

In [95]:
len(full_df_list), len(no_duplicates)

(6948, 5581)

In [97]:
no_duplicates.iloc[missing_indexes]

,titel,artiest
208,Whole Lotta Love,Led Zeppelin
215,Layla,Derek & The Dominos
363,Still Got The Blues,Gary Moore
395,Limburg,Rowwen Hèze
564,Eight Miles High,Byrds
...,...,...
51650,Whisky In The Jar (Albumversie),Thin Lizzy
51809,Just The Two Of Us (Albumversie),"Bill Withers & Grover Washington, Jr."
51964,Teardrops (Albumversie),Womack & Womack
51993,Fame (Albumversie),David Bowie


In [99]:
track_name, artist_name = no_duplicates.iloc[missing_indexes[0]]
data = get_metadata(track_name, artist_name)
data

{'error': 6, 'message': 'Track not found', 'links': []}

In [100]:
full_df_including_meta = pd.DataFrame(full_df_list)
full_df_including_meta

,duration,album,tags,artist,song
0,355000,A Night at the Opera,"[classic rock, rock, Queen]",Queen,Bohemian Rhapsody
1,360000,Hell Freezes Over,"[classic rock, rock, 70s]",Eagles,Hotel California
2,1010000,Deepest Purple: The Very Best of Deep Purple,"[classic rock, hard rock, rock]",Deep Purple,Child In Time
3,667000,Led Zeppelin IV,"[classic rock, rock, Led Zeppelin]",Led Zeppelin,Stairway To Heaven
4,332000,Bat Out Of Hell,"[classic rock, rock, Meat Loaf]",Meat Loaf,Paradise By The Dashboard Light
...,...,...,...,...,...
6943,241000,"Dagen Van Gras, Dagen Van Stro","[dutch, Nederlandstalig, Spinvis]",Spinvis,Ik Wil Alleen Maar Zwemmen
6944,0,None,[],Pink Floyd,Run Like Hell (Albumversie)
6945,0,None,"[Nederlandstalig, pop, Nederpop]",Acda en De Munnik,Als Je Bij Me Weggaat
6946,320000,Communiqué,"[rock, classic rock, dire straits]",Dire Straits,Once Upon A Time In The West


In [101]:
no_dups = full_df_including_meta.drop_duplicates(subset=["artist", "song"])
len(no_duplicates), len(no_dups)

(5581, 5514)

In [102]:
no_dups.to_csv("data/song_metadata.csv", index=False)